# Prototype
A Notebook to understand how to structure NCERT and other study material data into structured data for langchain.

Objectives:
- How to Sructure Study Material and NCERT Data
- How to give long term memory to LLM to make sure the LLM can Answer from its past experiece too

## Imports and Neccesary Variables
We will need Langchain library to connect Our Knowledge base to the LLM.Tools like LLMChain and ConversationChain will provide short-term memory and links to other tools.We will use CoversationBufferWindowMemory wih k=5 to keep only last 5 interactions since the Model has a Long term memory now.

In [1]:
!pip install langchain nltk bs4 requests fake_useragent
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from getpass import getpass

OPENAI_API_KEY = getpass("Open AI API Key: ")

Open AI API Key: ········


## Installing Various dataset of words

In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [66]:
from fake_useragent import UserAgent
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from urllib.parse import *
import requests
import heapq
import nltk
import json
import re

s = requests.Session()
s.faker = UserAgent()
s.headers = {
    'User-Agent': s.faker['chrome']
}

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

import string

# Function to extract tags from the content
def extract_tags(content, n_tags=5):
    # Tokenize the content into words
    tokens = word_tokenize(content.lower())

    # Remove stopwords from the tokens
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Calculate word frequencies
    fdist = FreqDist(filtered_tokens)

    # Extract the most common tokens as tags
    tags = set([token for token, _ in fdist.most_common(n_tags) if token not in string.punctuation])
    return tags

import math

def calculate_tfidf(content):
    # Tokenize the content into words
    tokens = word_tokenize(content.lower())

    # Remove stopwords from the tokens
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

    # Calculate term frequencies
    term_frequencies = nltk.FreqDist(filtered_tokens)
    max_frequency = max(term_frequencies.values())

    # Calculate document frequency
    document_frequency = nltk.FreqDist(filtered_tokens)

    # Calculate TF-IDF scores
    tfidf_scores = {}
    for token in filtered_tokens:
        tf = term_frequencies[token] / max_frequency
        idf = math.log(len(filtered_tokens) / (1 + document_frequency[token]))
        tfidf_scores[token] = tf * idf

    return tfidf_scores

# Example usage
content = "All living organisms have certain common characteristics such as breathing, growing, requiring nutrition, producing offspring, responding to stimuli, etc. There are certain vital processes that maintain homeostasis and proper functioning of the body, they are called life processes."

tfidf_scores = calculate_tfidf(content)
print(tfidf_scores)


def summarize(article):
    sentence_list = nltk.sent_tokenize(article.replace("\n",''))
    stopwrds = stopwords.words("english")
    word_frequencies = {}
    for word in nltk.word_tokenize(article):
        if word not in stopwrds:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    return str(summary)

{'living': 1.2628643221541278, 'organisms': 1.2628643221541278, 'certain': 2.120263536200091, 'common': 1.2628643221541278, 'characteristics': 1.2628643221541278, 'breathing': 1.2628643221541278, 'growing': 1.2628643221541278, 'requiring': 1.2628643221541278, 'nutrition': 1.2628643221541278, 'producing': 1.2628643221541278, 'offspring': 1.2628643221541278, 'responding': 1.2628643221541278, 'stimuli': 1.2628643221541278, 'etc': 1.2628643221541278, 'vital': 1.2628643221541278, 'processes': 2.120263536200091, 'maintain': 1.2628643221541278, 'homeostasis': 1.2628643221541278, 'proper': 1.2628643221541278, 'functioning': 1.2628643221541278, 'body': 1.2628643221541278, 'called': 1.2628643221541278, 'life': 1.2628643221541278}


## Get the data and structure it
Get data from various sources like NCERT books, and Online resources and structure it for the database.
The structured data will be then uploaded to any SQL databse and then be used as a KnowledgeBase for the Chat